In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
benf=pd.read_csv('../data/Train_Beneficiarydata-1542865627584.csv')
benf.head(10)
#dob- date of birth
#dod- date of death- missing value might endicate the person is alive
#dentaldiseaseind- often require expensive treatments 
#NoOfMonths_PartACovered- number of months the beneficiary was covered under part A: Submit claims during months when the patient was NOT covered
### financial coverage features:
#IPAnnualReimbursementAmt-> total amount of money medicalcare provided for inpatients(hospital stays, surgeries, emergency admissions, long-term recovery stays) services
#IPAnnualDeductibleAmt-> total amount the patient paid themselves (deductibles) for inpatient services
#OPAnnualReimbursementAmt-> total amount of money medicalcare provided for outpatient (doctor visits, lab tests, therapies, preventive care) services
#OPAnnualDeductibleAmt-> total amount the patient paid themselves (deductibles)

In [ ]:
inpatient=pd.read_csv('../data/Train_Inpatientdata-1542865627584.csv')
inpatient.head(10)
#DischargeDt-Should align with ClaimEndDt. used to compute length of hospital stay

In [ ]:
outpatient=pd.read_csv('../data/Train_Outpatientdata-1542865627584.csv')
outpatient.head(10)
#ClmAdmitDiagnosisCode-Primary diagnosis code when the outpatient service was requested

In [ ]:
benf.drop_duplicates()
inpatient.drop_duplicates()
outpatient.drop_duplicates()

In [ ]:
#load test
benf_test=pd.read_csv('../data/Test_Beneficiarydata-1542969243754.csv')
inpatient_test=pd.read_csv('../data/Test_Inpatientdata-1542969243754.csv')  
outpatient_test=pd.read_csv('../data/Test_Outpatientdata-1542969243754.csv')
test_id=pd.read_csv('../data/Test-1542969243754.csv')
benf_test.drop_duplicates()
inpatient_test.drop_duplicates()
outpatient_test.drop_duplicates()

In [ ]:
#Beneficiary Table

# Beneficiary Table
benf['DOB'] = pd.to_datetime(benf['DOB'], errors='coerce')
benf['DOD'] = pd.to_datetime(benf['DOD'], errors='coerce')

#add age attribute and is alive attribute
benf['is_alive'] = benf['DOD'].isna().astype(int)
benf['age'] = ((benf['DOD'].fillna(pd.Timestamp.today()) - benf['DOB']).dt.days / 365.25).astype(int)

# Check result
benf[['DOB', 'DOD', 'is_alive', 'age']].head()

#testing:

benf_test['DOB'] = pd.to_datetime(benf_test['DOB'], errors='coerce')
benf_test['DOD'] = pd.to_datetime(benf_test['DOD'], errors='coerce')

#add age attribute and is alive attribute
benf_test['is_alive'] = benf_test['DOD'].isna().astype(int)
benf_test['age'] = ((benf_test['DOD'].fillna(pd.Timestamp.today()) - benf_test['DOB']).dt.days / 365.25).astype(int)

# Check result
benf_test[['DOB', 'DOD', 'is_alive', 'age']].head()








In [ ]:
#inpatient
date_cols = inpatient.columns[inpatient.columns.str.contains('Dt|Date', case=False)]
for col in date_cols:
    inpatient[col] = pd.to_datetime(inpatient[col], errors='coerce')
inpatient.isnull().sum()
#test:
date_cols = inpatient_test.columns[inpatient_test.columns.str.contains('Dt|Date', case=False)]
for col in date_cols:
    inpatient_test[col] = pd.to_datetime(inpatient_test[col], errors='coerce')
inpatient_test.isnull().sum()

In [ ]:
# List of diagnosis and procedure columns
diagnosis_cols = [f'ClmDiagnosisCode_{i}' for i in range(1, 10)]
procedure_cols = [f'ClmProcedureCode_{i}' for i in range(1, 6)]

# Encode diagnosis codes: 1 if present, 0 if null
for col in diagnosis_cols:
    inpatient[col + '_present'] = inpatient[col].notna().astype(int)

# Encode procedure codes: 1 if present, 0 if null
for col in procedure_cols:
    inpatient[col + '_present'] = inpatient[col].notna().astype(int)

# Optional: compute total diagnoses and total procedures per claim
inpatient['num_diagnoses'] = inpatient[[col + '_present' for col in diagnosis_cols]].sum(axis=1)
inpatient['num_procedures'] = inpatient[[col + '_present' for col in procedure_cols]].sum(axis=1)

# Quick check
inpatient[['num_diagnoses','num_procedures'] + [col + '_present' for col in diagnosis_cols[:3]] + [col + '_present' for col in procedure_cols[:3]]].head()



#test:


# List of diagnosis and procedure columns
diagnosis_cols = [f'ClmDiagnosisCode_{i}' for i in range(1, 10)]
procedure_cols = [f'ClmProcedureCode_{i}' for i in range(1, 6)]

# Encode diagnosis codes: 1 if present, 0 if null
for col in diagnosis_cols:
    inpatient_test[col + '_present'] = inpatient_test[col].notna().astype(int)

# Encode procedure codes: 1 if present, 0 if null
for col in procedure_cols:
    inpatient_test[col + '_present'] = inpatient_test[col].notna().astype(int)

# Optional: compute total diagnoses and total procedures per claim
inpatient_test['num_diagnoses'] = inpatient_test[[col + '_present' for col in diagnosis_cols]].sum(axis=1)
inpatient_test['num_procedures'] = inpatient_test[[col + '_present' for col in procedure_cols]].sum(axis=1)

# Quick check
inpatient_test[['num_diagnoses','num_procedures'] + [col + '_present' for col in diagnosis_cols[:3]] + [col + '_present' for col in procedure_cols[:3]]].head()




In [ ]:

today = pd.Timestamp.today()

# safe helpers (fill missing numeric amounts to 0)
inpatient['DeductibleAmtPaid'] = inpatient['DeductibleAmtPaid'].fillna(0)
inpatient['InscClaimAmtReimbursed'] = inpatient['InscClaimAmtReimbursed'].fillna(0)

# Payment flags
inpatient['patient_paid'] = (inpatient['DeductibleAmtPaid'] > 0).astype(int)
inpatient['insurer_paid'] = (inpatient['InscClaimAmtReimbursed'] > 0).astype(int)
inpatient['any_payment'] = ((inpatient['patient_paid'] == 1) | (inpatient['insurer_paid'] == 1)).astype(int)

# Duration features (in days). Use NaN-safe computations.
inpatient['los_days'] = (inpatient['DischargeDt'] - inpatient['AdmissionDt']).dt.days
inpatient['claim_length_days'] = (inpatient['ClaimEndDt'] - inpatient['ClaimStartDt']).dt.days

# Avoid negative/infinite weirdness: keep NaNs where dates missing
# Basic anomaly flags (strong anomalies)
inpatient['ClaimStart_after_ClaimEnd'] = ((inpatient['ClaimStartDt'].notna() & inpatient['ClaimEndDt'].notna()) &
                                         (inpatient['ClaimStartDt'] > inpatient['ClaimEndDt'])).astype(int)

inpatient['Admission_after_Discharge'] = ((inpatient['AdmissionDt'].notna() & inpatient['DischargeDt'].notna()) &
                                          (inpatient['AdmissionDt'] > inpatient['DischargeDt'])).astype(int)

inpatient['Future_dates'] = (inpatient[['ClaimStartDt','ClaimEndDt','AdmissionDt','DischargeDt']].gt(today)
                             .any(axis=1)).astype(int)

# Potential anomalies (sometimes legitimate)
inpatient['ClaimStart_before_Admission'] = ((inpatient['ClaimStartDt'].notna() & inpatient['AdmissionDt'].notna()) &
                                           (inpatient['ClaimStartDt'] < inpatient['AdmissionDt'])).astype(int)

inpatient['ClaimEnd_after_Discharge'] = ((inpatient['ClaimEndDt'].notna() & inpatient['DischargeDt'].notna()) &
                                         (inpatient['ClaimEndDt'] > inpatient['DischargeDt'])).astype(int)

# Conditional anomaly features using payment info:
# - If claim extends after discharge AND no patient payment, mark more suspicious
inpatient['ClaimEnd_after_Discharge_no_patient_payment'] = (
    (inpatient['ClaimEnd_after_Discharge'] == 1) & (inpatient['patient_paid'] == 0)
).astype(int)

# - If claim starts before admission AND there's no payment (insurer nor patient), mark suspicious
inpatient['ClaimStart_before_Admission_no_payment'] = (
    (inpatient['ClaimStart_before_Admission'] == 1) & (inpatient['any_payment'] == 0)
).astype(int)

# Length-based anomalies:
# - If claim duration is much larger than LOS (e.g., claim_length > 2 * los_days), suspicious.
#   Requires both claim_length and los_days to be non-null and los_days >= 0.
inpatient['claim_length_vs_los_ratio'] = None
mask_valid_lengths = inpatient['claim_length_days'].notna() & inpatient['los_days'].notna() & (inpatient['los_days'] >= 0)
inpatient.loc[mask_valid_lengths, 'claim_length_vs_los_ratio'] = (
    inpatient.loc[mask_valid_lengths, 'claim_length_days'] / inpatient.loc[mask_valid_lengths, 'los_days'].replace({0: 1})
)
inpatient['claim_length_vs_los_ratio'] = inpatient['claim_length_vs_los_ratio'].astype(float)

# Flag when ratio is large (use >2 as an initial heuristic)
inpatient['claim_length_much_greater_than_los'] = ((inpatient['claim_length_vs_los_ratio'] > 2) & mask_valid_lengths).astype(int)

# Build a weighted date-anomaly score:
# weight strong anomalies higher (3), conditional suspicious (2), potential anomalies (1)
inpatient['date_anomaly_score'] = (
      3 * inpatient['ClaimStart_after_ClaimEnd'].fillna(0)
    + 3 * inpatient['Admission_after_Discharge'].fillna(0)
    + 3 * inpatient['Future_dates'].fillna(0)
    + 2 * inpatient['ClaimEnd_after_Discharge_no_patient_payment'].fillna(0)
    + 1 * inpatient['ClaimStart_before_Admission_no_payment'].fillna(0)
    + 2 * inpatient['claim_length_much_greater_than_los'].fillna(0)
)

# For transparency, also keep a simple count of distinct date issues (unweighted)
date_issue_cols = ['ClaimStart_after_ClaimEnd','Admission_after_Discharge','Future_dates',
                   'ClaimEnd_after_Discharge','ClaimStart_before_Admission']
inpatient['date_issue_count'] = inpatient[date_issue_cols].sum(axis=1)

# Quick summary for review
summary = inpatient[['ClaimStart_after_ClaimEnd','Admission_after_Discharge','ClaimStart_before_Admission',
                     'ClaimEnd_after_Discharge','ClaimEnd_after_Discharge_no_patient_payment',
                     'ClaimStart_before_Admission_no_payment','Future_dates',
                     'claim_length_much_greater_than_los','date_anomaly_score','date_issue_count']].sum()
print("Feature summary (counts / sums):")
print(summary)

# Example: show top suspicious rows by score
print("\nTop suspicious claims by date_anomaly_score:")
print(inpatient.sort_values('date_anomaly_score', ascending=False)
                .loc[:, ['ClaimID','Provider','BeneID','date_anomaly_score','date_issue_count',
                         'ClaimStartDt','ClaimEndDt','AdmissionDt','DischargeDt',
                         'patient_paid','insurer_paid','InscClaimAmtReimbursed','DeductibleAmtPaid']]
                .head(20))




#test:



today = pd.Timestamp.today()

# safe helpers (fill missing numeric amounts to 0)
inpatient_test['DeductibleAmtPaid'] = inpatient_test['DeductibleAmtPaid'].fillna(0)
inpatient_test['InscClaimAmtReimbursed'] = inpatient_test['InscClaimAmtReimbursed'].fillna(0)

# Payment flags
inpatient_test['patient_paid'] = (inpatient_test['DeductibleAmtPaid'] > 0).astype(int)
inpatient_test['insurer_paid'] = (inpatient_test['InscClaimAmtReimbursed'] > 0).astype(int)
inpatient_test['any_payment'] = ((inpatient_test['patient_paid'] == 1) | (inpatient_test['insurer_paid'] == 1)).astype(int)

# Duration features (in days). Use NaN-safe computations.
inpatient_test['los_days'] = (inpatient_test['DischargeDt'] - inpatient_test['AdmissionDt']).dt.days
inpatient_test['claim_length_days'] = (inpatient_test['ClaimEndDt'] - inpatient_test['ClaimStartDt']).dt.days

# Avoid negative/infinite weirdness: keep NaNs where dates missing
# Basic anomaly flags (strong anomalies)
inpatient_test['ClaimStart_after_ClaimEnd'] = ((inpatient_test['ClaimStartDt'].notna() & inpatient_test['ClaimEndDt'].notna()) &
                                         (inpatient_test['ClaimStartDt'] > inpatient_test['ClaimEndDt'])).astype(int)

inpatient_test['Admission_after_Discharge'] = ((inpatient_test['AdmissionDt'].notna() & inpatient_test['DischargeDt'].notna()) &
                                          (inpatient_test['AdmissionDt'] > inpatient_test['DischargeDt'])).astype(int)

inpatient_test['Future_dates'] = (inpatient_test[['ClaimStartDt','ClaimEndDt','AdmissionDt','DischargeDt']].gt(today)
                             .any(axis=1)).astype(int)

# Potential anomalies (sometimes legitimate)
inpatient_test['ClaimStart_before_Admission'] = ((inpatient_test['ClaimStartDt'].notna() & inpatient_test['AdmissionDt'].notna()) &
                                           (inpatient_test['ClaimStartDt'] < inpatient_test['AdmissionDt'])).astype(int)

inpatient_test['ClaimEnd_after_Discharge'] = ((inpatient_test['ClaimEndDt'].notna() & inpatient_test['DischargeDt'].notna()) &
                                         (inpatient_test['ClaimEndDt'] > inpatient_test['DischargeDt'])).astype(int)

# Conditional anomaly features using payment info:
# - If claim extends after discharge AND no patient payment, mark more suspicious
inpatient_test['ClaimEnd_after_Discharge_no_patient_payment'] = (
    (inpatient_test['ClaimEnd_after_Discharge'] == 1) & (inpatient_test['patient_paid'] == 0)
).astype(int)

# - If claim starts before admission AND there's no payment (insurer nor patient), mark suspicious
inpatient_test['ClaimStart_before_Admission_no_payment'] = (
    (inpatient_test['ClaimStart_before_Admission'] == 1) & (inpatient_test['any_payment'] == 0)
).astype(int)

# Length-based anomalies:
# - If claim duration is much larger than LOS (e.g., claim_length > 2 * los_days), suspicious.
#   Requires both claim_length and los_days to be non-null and los_days >= 0.
inpatient_test['claim_length_vs_los_ratio'] = None
mask_valid_lengths = inpatient_test['claim_length_days'].notna() & inpatient_test['los_days'].notna() & (inpatient_test['los_days'] >= 0)
inpatient_test.loc[mask_valid_lengths, 'claim_length_vs_los_ratio'] = (
    inpatient_test.loc[mask_valid_lengths, 'claim_length_days'] / inpatient_test.loc[mask_valid_lengths, 'los_days'].replace({0: 1})
)
inpatient_test['claim_length_vs_los_ratio'] = inpatient_test['claim_length_vs_los_ratio'].astype(float)

# Flag when ratio is large (use >2 as an initial heuristic)
inpatient_test['claim_length_much_greater_than_los'] = ((inpatient_test['claim_length_vs_los_ratio'] > 2) & mask_valid_lengths).astype(int)

# Build a weighted date-anomaly score:
# weight strong anomalies higher (3), conditional suspicious (2), potential anomalies (1)
inpatient_test['date_anomaly_score'] = (
      3 * inpatient_test['ClaimStart_after_ClaimEnd'].fillna(0)
    + 3 * inpatient_test['Admission_after_Discharge'].fillna(0)
    + 3 * inpatient_test['Future_dates'].fillna(0)
    + 2 * inpatient_test['ClaimEnd_after_Discharge_no_patient_payment'].fillna(0)
    + 1 * inpatient_test['ClaimStart_before_Admission_no_payment'].fillna(0)
    + 2 * inpatient_test['claim_length_much_greater_than_los'].fillna(0)
)

# For transparency, also keep a simple count of distinct date issues (unweighted)
date_issue_cols = ['ClaimStart_after_ClaimEnd','Admission_after_Discharge','Future_dates',
                   'ClaimEnd_after_Discharge','ClaimStart_before_Admission']
inpatient_test['date_issue_count'] = inpatient_test[date_issue_cols].sum(axis=1)

# Quick summary for review
summary = inpatient_test[['ClaimStart_after_ClaimEnd','Admission_after_Discharge','ClaimStart_before_Admission',
                     'ClaimEnd_after_Discharge','ClaimEnd_after_Discharge_no_patient_payment',
                     'ClaimStart_before_Admission_no_payment','Future_dates',
                     'claim_length_much_greater_than_los','date_anomaly_score','date_issue_count']].sum()
print("Feature summary (counts / sums):")
print(summary)

# Example: show top suspicious rows by score
print("\nTop suspicious claims by date_anomaly_score:")
print(inpatient_test.sort_values('date_anomaly_score', ascending=False)
                .loc[:, ['ClaimID','Provider','BeneID','date_anomaly_score','date_issue_count',
                         'ClaimStartDt','ClaimEndDt','AdmissionDt','DischargeDt',
                         'patient_paid','insurer_paid','InscClaimAmtReimbursed','DeductibleAmtPaid']]
                .head(20))










In [ ]:
# For each feature (column), calculate % of missing values
for col in inpatient.columns:
    missing_pct = inpatient[col].isnull().sum() / len(inpatient) * 100
    print(f"{col}: {missing_pct:.2f}% missing")

#test:

for col in inpatient_test.columns:
    missing_pct = inpatient_test[col].isnull().sum() / len(inpatient_test) * 100
    print(f"{col}: {missing_pct:.2f}% missing")


In [ ]:
#inpatient
#1. encode physcians as either 1(present) or 0(absent) for each claim
#2. count frequency of appearance of each physcians
#3. sum number of physcians per person 


# Columns for physicians
phys_cols = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']

# 1️ Frequency Encoding

# Count how many times each physician ID appears across all three columns
phys_freq = pd.concat([inpatient[col] for col in phys_cols]).value_counts()

# Map the frequency back to each physician column, missing IDs get 0
for col in phys_cols:
    inpatient[col + '_freq'] = inpatient[col].map(phys_freq).fillna(0)


#2. 0/1 Presence Encoding
# Create new columns indicating presence of a physician ID
for col in phys_cols:
    inpatient[col + '_presence'] = inpatient[col].notna().astype(int)


# 3 Number of Physicians per claim

# Sum the presence columns to get total number of physicians per claim
inpatient['num_physicians'] = inpatient[[col + '_presence' for col in phys_cols]].sum(axis=1)


print(inpatient[[col + '_freq' for col in phys_cols] +
               [col + '_presence' for col in phys_cols] +
               ['num_physicians']].head())





#test:


#inpatient
#1. encode physcians as either 1(present) or 0(absent) for each claim
#2. count frequency of appearance of each physcians
#3. sum number of physcians per person 


# Columns for physicians
phys_cols = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']

# 1️ Frequency Encoding

# Count how many times each physician ID appears across all three columns
phys_freq = pd.concat([inpatient_test[col] for col in phys_cols]).value_counts()

# Map the frequency back to each physician column, missing IDs get 0
for col in phys_cols:
    inpatient_test[col + '_freq'] = inpatient_test[col].map(phys_freq).fillna(0)


#2. 0/1 Presence Encoding
# Create new columns indicating presence of a physician ID
for col in phys_cols:
    inpatient_test[col + '_presence'] = inpatient_test[col].notna().astype(int)


# 3 Number of Physicians per claim

# Sum the presence columns to get total number of physicians per claim
inpatient_test['num_physicians'] = inpatient_test[[col + '_presence' for col in phys_cols]].sum(axis=1)


print(inpatient_test[[col + '_freq' for col in phys_cols] +
               [col + '_presence' for col in phys_cols] +
               ['num_physicians']].head())







In [ ]:
# diagnosis columns
diag_cols = [f'ClmDiagnosisCode_{i}' for i in range(1, 11)]

# 0. Ensure numeric payments and claim_length_days exist
inpatient['DeductibleAmtPaid'] = inpatient['DeductibleAmtPaid'].fillna(0)
inpatient['InscClaimAmtReimbursed'] = inpatient['InscClaimAmtReimbursed'].fillna(0)
inpatient['total_payment'] = inpatient['DeductibleAmtPaid'] + inpatient['InscClaimAmtReimbursed']

# ensure claim_length_days exists and use at least 1 day to avoid div by 0
if 'claim_length_days' not in inpatient.columns:
    inpatient['claim_length_days'] = (inpatient['ClaimEndDt'] - inpatient['ClaimStartDt']).dt.days
inpatient['claim_length_days'] = inpatient['claim_length_days'].fillna(0).astype(float)
inpatient['claim_length_days_nonzero'] = inpatient['claim_length_days'].apply(lambda x: x if x >= 1 else 1.0)

# 1. payment per day (ppd) for each claim
inpatient['claim_payment_ppd'] = inpatient['total_payment'] / inpatient['claim_length_days_nonzero']

# 2. build long table of (ClaimID, diag_code, claim_payment_ppd)
# keep only valid diagnosis codes (non-null and not placeholder like 'Unknown' or '')
long = (inpatient[['ClaimID'] + diag_cols + ['claim_payment_ppd']]
        .set_index('ClaimID')
        .reset_index())

# melt
melted = long.melt(id_vars=['ClaimID','claim_payment_ppd'], value_vars=diag_cols,
                   var_name='diag_slot', value_name='diag_code')

# clean diag_code: drop NaN and empty/Unknown placeholders
melted['diag_code'] = melted['diag_code'].astype(str).str.strip()
# treat common placeholders as missing
placeholders = {'nan', 'none', 'unknown', 'na', 'nan.', ''} 
melted = melted[~melted['diag_code'].str.lower().isin(placeholders)]
melted = melted[~melted['diag_code'].isin(['nan', 'None'])]

# 3. compute diag-level stats: count, mean_ppd, median_ppd
diag_stats = (melted.groupby('diag_code')['claim_payment_ppd']
                    .agg(['count', 'mean', 'median'])
                    .rename(columns={'mean':'mean_ppd', 'median':'median_ppd'})
                    .reset_index())

# 4. smoothing (shrink mean toward global mean) to stabilize rare codes
global_mean = melted['claim_payment_ppd'].median()  # median as robust global center
k = 10.0  # prior strength (tuneable)
diag_stats['smoothed_mean_ppd'] = (diag_stats['count'] * diag_stats['mean_ppd'] + k * global_mean) / (diag_stats['count'] + k)

# 5. map stats back to each diag slot in inpatient
# build mapping dicts
smoothed_map = diag_stats.set_index('diag_code')['smoothed_mean_ppd'].to_dict()
median_map = diag_stats.set_index('diag_code')['median_ppd'].to_dict()
count_map = diag_stats.set_index('diag_code')['count'].to_dict()

# For each diag slot, create mapped expected_ppd, median_ppd, count
for col in diag_cols:
    inpatient[col + '_diag_exp_ppd'] = inpatient[col].map(smoothed_map).fillna(0.0)
    inpatient[col + '_diag_median_ppd'] = inpatient[col].map(median_map).fillna(0.0)
    inpatient[col + '_diag_count'] = inpatient[col].map(count_map).fillna(0).astype(int)

# 6. compute claim-level aggregated diagnosis expectations
# consider only diag slots with nonzero expected (i.e., present in mapping)
exp_cols = [c + '_diag_exp_ppd' for c in diag_cols]
med_cols = [c + '_diag_median_ppd' for c in diag_cols]
cnt_cols = [c + '_diag_count' for c in diag_cols]

# mean expected ppd across present diagnosis codes (ignore zeros)
inpatient['diag_expected_mean_ppd'] = inpatient[exp_cols].replace(0, np.nan).mean(axis=1).fillna(0.0)
inpatient['diag_expected_median_ppd'] = inpatient[med_cols].replace(0, np.nan).median(axis=1).fillna(0.0)
inpatient['diag_expected_max_ppd'] = inpatient[exp_cols].max(axis=1).fillna(0.0)
inpatient['diag_expected_sum_ppd'] = inpatient[exp_cols].sum(axis=1).fillna(0.0)
inpatient['diag_present_count'] = (inpatient[exp_cols] > 0).sum(axis=1)

# 7. ratio features: actual claim_ppd divided by expected
# if expected mean is 0 (no known diag), set ratio to NaN (or large sentinel); here use NaN
inpatient['claim_ppd_to_diag_expected_mean'] = np.where(
    inpatient['diag_expected_mean_ppd'] > 0,
    inpatient['claim_payment_ppd'] / inpatient['diag_expected_mean_ppd'],
    np.nan
)

inpatient['claim_ppd_to_diag_expected_max'] = np.where(
    inpatient['diag_expected_max_ppd'] > 0,
    inpatient['claim_payment_ppd'] / inpatient['diag_expected_max_ppd'],
    np.nan
)

# 8. rare-diagnosis flags: any diag in claim has count < threshold
rare_threshold = 5
inpatient['claim_has_rare_diag'] = (inpatient[cnt_cols] > 0).apply(lambda row: any(row < rare_threshold), axis=1).astype(int)

# 9. diagnostic expected vs actual gap (absolute and percent)
inpatient['diag_expected_sum_ppd'] = inpatient['diag_expected_sum_ppd'].replace(0, np.nan)
inpatient['expected_vs_actual_gap_ppd'] = inpatient['claim_payment_ppd'] - inpatient['diag_expected_mean_ppd']  # absolute gap
inpatient['expected_vs_actual_pct_ppd'] = inpatient['claim_payment_ppd'] / inpatient['diag_expected_mean_ppd'].replace({0: np.nan})

# 10. clean up temporary columns if desired (optional)
# keep final features: diag_expected_mean_ppd, diag_expected_median_ppd, diag_expected_max_ppd,
# diag_expected_sum_ppd, diag_present_count, claim_ppd_to_diag_expected_mean, claim_has_rare_diag, expected_vs_actual_gap_ppd
final_diag_features = [
    'diag_expected_mean_ppd', 'diag_expected_median_ppd', 'diag_expected_max_ppd',
    'diag_expected_sum_ppd', 'diag_present_count', 'claim_ppd_to_diag_expected_mean',
    'claim_ppd_to_diag_expected_max', 'claim_has_rare_diag', 'expected_vs_actual_gap_ppd',
    'expected_vs_actual_pct_ppd'
]

# (optional) print a quick summary
print("Diagnosis-payment feature creation complete.")
print("Number of unique diagnosis codes modeled:", len(diag_stats))
print(diag_stats[['diag_code','count','median_ppd','smoothed_mean_ppd']].sort_values('count', ascending=False).head(10))





#test:


# diagnosis columns
diag_cols = [f'ClmDiagnosisCode_{i}' for i in range(1, 11)]

# 0. Ensure numeric payments and claim_length_days exist
inpatient_test['DeductibleAmtPaid'] = inpatient_test['DeductibleAmtPaid'].fillna(0)
inpatient_test['InscClaimAmtReimbursed'] = inpatient_test['InscClaimAmtReimbursed'].fillna(0)
inpatient_test['total_payment'] = inpatient_test['DeductibleAmtPaid'] + inpatient_test['InscClaimAmtReimbursed']

# ensure claim_length_days exists and use at least 1 day to avoid div by 0
if 'claim_length_days' not in inpatient_test.columns:
    inpatient_test['claim_length_days'] = (inpatient_test['ClaimEndDt'] - inpatient_test['ClaimStartDt']).dt.days
inpatient_test['claim_length_days'] = inpatient_test['claim_length_days'].fillna(0).astype(float)
inpatient_test['claim_length_days_nonzero'] = inpatient_test['claim_length_days'].apply(lambda x: x if x >= 1 else 1.0)

# 1. payment per day (ppd) for each claim
inpatient_test['claim_payment_ppd'] = inpatient_test['total_payment'] / inpatient_test['claim_length_days_nonzero']

# 2. build long table of (ClaimID, diag_code, claim_payment_ppd)
# keep only valid diagnosis codes (non-null and not placeholder like 'Unknown' or '')
long = (inpatient_test[['ClaimID'] + diag_cols + ['claim_payment_ppd']]
        .set_index('ClaimID')
        .reset_index())

# melt
melted = long.melt(id_vars=['ClaimID','claim_payment_ppd'], value_vars=diag_cols,
                   var_name='diag_slot', value_name='diag_code')

# clean diag_code: drop NaN and empty/Unknown placeholders
melted['diag_code'] = melted['diag_code'].astype(str).str.strip()
# treat common placeholders as missing
placeholders = {'nan', 'none', 'unknown', 'na', 'nan.', ''} 
melted = melted[~melted['diag_code'].str.lower().isin(placeholders)]
melted = melted[~melted['diag_code'].isin(['nan', 'None'])]

# 3. compute diag-level stats: count, mean_ppd, median_ppd
diag_stats = (melted.groupby('diag_code')['claim_payment_ppd']
                    .agg(['count', 'mean', 'median'])
                    .rename(columns={'mean':'mean_ppd', 'median':'median_ppd'})
                    .reset_index())

# 4. smoothing (shrink mean toward global mean) to stabilize rare codes
global_mean = melted['claim_payment_ppd'].median()  # median as robust global center
k = 10.0  # prior strength (tuneable)
diag_stats['smoothed_mean_ppd'] = (diag_stats['count'] * diag_stats['mean_ppd'] + k * global_mean) / (diag_stats['count'] + k)

# 5. map stats back to each diag slot in inpatient_test
# build mapping dicts
smoothed_map = diag_stats.set_index('diag_code')['smoothed_mean_ppd'].to_dict()
median_map = diag_stats.set_index('diag_code')['median_ppd'].to_dict()
count_map = diag_stats.set_index('diag_code')['count'].to_dict()

# For each diag slot, create mapped expected_ppd, median_ppd, count
for col in diag_cols:
    inpatient_test[col + '_diag_exp_ppd'] = inpatient_test[col].map(smoothed_map).fillna(0.0)
    inpatient_test[col + '_diag_median_ppd'] = inpatient_test[col].map(median_map).fillna(0.0)
    inpatient_test[col + '_diag_count'] = inpatient_test[col].map(count_map).fillna(0).astype(int)

# 6. compute claim-level aggregated diagnosis expectations
# consider only diag slots with nonzero expected (i.e., present in mapping)
exp_cols = [c + '_diag_exp_ppd' for c in diag_cols]
med_cols = [c + '_diag_median_ppd' for c in diag_cols]
cnt_cols = [c + '_diag_count' for c in diag_cols]

# mean expected ppd across present diagnosis codes (ignore zeros)
inpatient_test['diag_expected_mean_ppd'] = inpatient_test[exp_cols].replace(0, np.nan).mean(axis=1).fillna(0.0)
inpatient_test['diag_expected_median_ppd'] = inpatient_test[med_cols].replace(0, np.nan).median(axis=1).fillna(0.0)
inpatient_test['diag_expected_max_ppd'] = inpatient_test[exp_cols].max(axis=1).fillna(0.0)
inpatient_test['diag_expected_sum_ppd'] = inpatient_test[exp_cols].sum(axis=1).fillna(0.0)
inpatient_test['diag_present_count'] = (inpatient_test[exp_cols] > 0).sum(axis=1)

# 7. ratio features: actual claim_ppd divided by expected
# if expected mean is 0 (no known diag), set ratio to NaN (or large sentinel); here use NaN
inpatient_test['claim_ppd_to_diag_expected_mean'] = np.where(
    inpatient_test['diag_expected_mean_ppd'] > 0,
    inpatient_test['claim_payment_ppd'] / inpatient_test['diag_expected_mean_ppd'],
    np.nan
)

inpatient_test['claim_ppd_to_diag_expected_max'] = np.where(
    inpatient_test['diag_expected_max_ppd'] > 0,
    inpatient_test['claim_payment_ppd'] / inpatient_test['diag_expected_max_ppd'],
    np.nan
)

# 8. rare-diagnosis flags: any diag in claim has count < threshold
rare_threshold = 5
inpatient_test['claim_has_rare_diag'] = (inpatient_test[cnt_cols] > 0).apply(lambda row: any(row < rare_threshold), axis=1).astype(int)

# 9. diagnostic expected vs actual gap (absolute and percent)
inpatient_test['diag_expected_sum_ppd'] = inpatient_test['diag_expected_sum_ppd'].replace(0, np.nan)
inpatient_test['expected_vs_actual_gap_ppd'] = inpatient_test['claim_payment_ppd'] - inpatient_test['diag_expected_mean_ppd']  # absolute gap
inpatient_test['expected_vs_actual_pct_ppd'] = inpatient_test['claim_payment_ppd'] / inpatient_test['diag_expected_mean_ppd'].replace({0: np.nan})

# 10. clean up temporary columns if desired (optional)
final_diag_features = [
    'diag_expected_mean_ppd', 'diag_expected_median_ppd', 'diag_expected_max_ppd',
    'diag_expected_sum_ppd', 'diag_present_count', 'claim_ppd_to_diag_expected_mean',
    'claim_ppd_to_diag_expected_max', 'claim_has_rare_diag', 'expected_vs_actual_gap_ppd',
    'expected_vs_actual_pct_ppd'
]

# (optional) print a quick summary
print("Diagnosis-payment feature creation complete.")
print("Number of unique diagnosis codes modeled:", len(diag_stats))
print(diag_stats[['diag_code','count','median_ppd','smoothed_mean_ppd']].sort_values('count', ascending=False).head(10))












In [ ]:
inpatient.head(5)
#what is next: handle outliers(top), encoding for provider and Physicians, check if there are any NAN left
#read about: Optional: Feature transformations, Log-transform payments, ratios to reduce skew, Scaling numeric features if using linear models
#Temporal / rolling features, Trend of payments or anomalies over time per provider
#Save preprocessing artifacts Mapping dictionaries for diag/payment, provider freq, scalers

In [ ]:
###outpatient
#1) Parse date columns (safe)
date_cols = outpatient.columns[outpatient.columns.str.contains('Dt|Date', case=False)]
for col in date_cols:
    outpatient[col] = pd.to_datetime(outpatient[col], errors='coerce')

today = pd.Timestamp.today()

# 2) Diagnosis / Procedure presence flags & counts
diag_cols = [f'ClmDiagnosisCode_{i}' for i in range(1, 11)]
procedure_cols = [f'ClmProcedureCode_{i}' for i in range(1, 7)]

# presence flags (1 if present, 0 if null/empty/placeholder)
placeholders = {'nan', 'none', 'unknown', 'na', '', 'nan.'}
for col in diag_cols:
    outpatient[col] = outpatient[col].astype(str).str.strip().replace({c: np.nan for c in placeholders})
    outpatient[col + '_present'] = outpatient[col].notna().astype(int)

for col in procedure_cols:
    outpatient[col] = outpatient[col].astype(str).str.strip().replace({c: np.nan for c in placeholders})
    outpatient[col + '_present'] = outpatient[col].notna().astype(int)

# totals per claim
outpatient['num_diagnoses'] = outpatient[[c + '_present' for c in diag_cols]].sum(axis=1)
outpatient['num_procedures'] = outpatient[[c + '_present' for c in procedure_cols]].sum(axis=1)

# 3) Safe numeric payments
outpatient['DeductibleAmtPaid'] = outpatient['DeductibleAmtPaid'].fillna(0).astype(float)
outpatient['InscClaimAmtReimbursed'] = outpatient['InscClaimAmtReimbursed'].fillna(0).astype(float)
outpatient['total_payment'] = outpatient['DeductibleAmtPaid'] + outpatient['InscClaimAmtReimbursed']

# payment flags
outpatient['patient_paid'] = (outpatient['DeductibleAmtPaid'] > 0).astype(int)
outpatient['insurer_paid'] = (outpatient['InscClaimAmtReimbursed'] > 0).astype(int)
outpatient['any_payment'] = ((outpatient['patient_paid'] == 1) | (outpatient['insurer_paid'] == 1)).astype(int)

# 4) Duration feature (days) — usually 0 for outpatient but keep it
outpatient['claim_duration_days'] = (outpatient['ClaimEndDt'] - outpatient['ClaimStartDt']).dt.days
# keep NaNs if dates missing; also a nonzero-safe version for division when needed
outpatient['claim_duration_days_nonzero'] = outpatient['claim_duration_days'].apply(lambda x: x if (pd.notna(x) and x >= 1) else 1.0)

# 5) Date anomalies relevant to outpatient
outpatient['ClaimStart_after_ClaimEnd'] = ((outpatient['ClaimStartDt'].notna() & outpatient['ClaimEndDt'].notna()) &
                                           (outpatient['ClaimStartDt'] > outpatient['ClaimEndDt'])).astype(int)

outpatient['ClaimStart_before_ClaimEnd'] = ((outpatient['ClaimStartDt'].notna() & outpatient['ClaimEndDt'].notna()) &
                                            (outpatient['ClaimStartDt'] < outpatient['ClaimEndDt'])).astype(int)  # mostly true

outpatient['Future_dates'] = (outpatient[['ClaimStartDt','ClaimEndDt']].gt(today).any(axis=1)).astype(int)

outpatient['ClaimStart_before_Now_but_End_missing'] = ((outpatient['ClaimStartDt'].notna() & outpatient['ClaimEndDt'].isna())).astype(int)

# conditional suspicious flags:
outpatient['ClaimStart_after_ClaimEnd_no_payment'] = ((outpatient['ClaimStart_after_ClaimEnd'] == 1) & (outpatient['any_payment'] == 0)).astype(int)
outpatient['Future_dates_no_payment'] = ((outpatient['Future_dates'] == 1) & (outpatient['any_payment'] == 0)).astype(int)

# 6) Physician features (same logic as inpatient)
phys_cols = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']

# freq encoding across all physician columns
phys_freq = pd.concat([outpatient[col] for col in phys_cols]).value_counts()
for col in phys_cols:
    outpatient[col + '_freq'] = outpatient[col].map(phys_freq).fillna(0).astype(int)
    outpatient[col + '_presence'] = outpatient[col].notna().astype(int)

outpatient['num_physicians'] = outpatient[[col + '_presence' for col in phys_cols]].sum(axis=1)

# 7) Diagnosis-level expected-payment modeling (smoothed)
# We'll map diag -> smoothed mean of total_payment for claims containing that diag.
# Build a long table of ClaimID, diag_code, claim_payment
long = outpatient[['ClaimID'] + diag_cols + ['total_payment']].set_index('ClaimID').reset_index()
melted = long.melt(id_vars=['ClaimID','total_payment'], value_vars=diag_cols,
                   var_name='diag_slot', value_name='diag_code')
# clean diag_code
melted['diag_code'] = melted['diag_code'].astype(str).str.strip()
melted = melted[~melted['diag_code'].str.lower().isin(placeholders)]
melted = melted[melted['diag_code'].notna()]

# compute diag-level stats
diag_stats = (melted.groupby('diag_code')['total_payment']
                    .agg(['count', 'mean', 'median'])
                    .rename(columns={'mean':'mean_payment', 'median':'median_payment'})
                    .reset_index())

# smoothing toward global median to stabilise rare codes
global_center = melted['total_payment'].median() if len(melted)>0 else 0.0
k = 10.0  # prior strength; tuneable
diag_stats['smoothed_mean_payment'] = (diag_stats['count'] * diag_stats['mean_payment'] + k * global_center) / (diag_stats['count'] + k)

# mapping dicts
smoothed_map = diag_stats.set_index('diag_code')['smoothed_mean_payment'].to_dict()
median_map = diag_stats.set_index('diag_code')['median_payment'].to_dict()
count_map = diag_stats.set_index('diag_code')['count'].to_dict()

# map back to outpatient per diag slot
for col in diag_cols:
    outpatient[col + '_diag_exp_payment'] = outpatient[col].map(smoothed_map).fillna(0.0)
    outpatient[col + '_diag_median_payment'] = outpatient[col].map(median_map).fillna(0.0)
    outpatient[col + '_diag_count'] = outpatient[col].map(count_map).fillna(0).astype(int)

# aggregate diag-derived expectations at claim-level
exp_cols = [c + '_diag_exp_payment' for c in diag_cols]
med_cols = [c + '_diag_median_payment' for c in diag_cols]
cnt_cols = [c + '_diag_count' for c in diag_cols]

outpatient['diag_expected_mean_payment'] = outpatient[exp_cols].replace(0, np.nan).mean(axis=1).fillna(0.0)
outpatient['diag_expected_median_payment'] = outpatient[med_cols].replace(0, np.nan).median(axis=1).fillna(0.0)
outpatient['diag_expected_max_payment'] = outpatient[exp_cols].max(axis=1).fillna(0.0)
outpatient['diag_expected_sum_payment'] = outpatient[exp_cols].sum(axis=1).fillna(0.0)
outpatient['diag_present_count'] = (outpatient[exp_cols] > 0).sum(axis=1)

# ratio features: actual total_payment vs expected
outpatient['claim_to_diag_expected_mean'] = np.where(
    outpatient['diag_expected_mean_payment'] > 0,
    outpatient['total_payment'] / outpatient['diag_expected_mean_payment'],
    np.nan
)

outpatient['claim_to_diag_expected_max'] = np.where(
    outpatient['diag_expected_max_payment'] > 0,
    outpatient['total_payment'] / outpatient['diag_expected_max_payment'],
    np.nan
)

# rare-diagnosis flag
rare_threshold = 5
outpatient['claim_has_rare_diag'] = (outpatient[cnt_cols] > 0).apply(lambda row: any(row < rare_threshold), axis=1).astype(int)

# expected vs actual gaps
outpatient['expected_vs_actual_gap'] = outpatient['total_payment'] - outpatient['diag_expected_mean_payment']
outpatient['expected_vs_actual_pct'] = outpatient['total_payment'] / outpatient['diag_expected_mean_payment'].replace({0: np.nan})

# 8) Summaries & quick checks
print("Outpatient date/anomaly summary:")
date_issue_cols = ['ClaimStart_after_ClaimEnd','Future_dates','ClaimStart_before_ClaimEnd','ClaimStart_after_ClaimEnd_no_payment','Future_dates_no_payment']
print(outpatient[date_issue_cols].sum())

print("\nTop suspicious outpatient claims by expected_vs_actual_gap:")
print(outpatient.sort_values('expected_vs_actual_gap', ascending=False)
                .loc[:, ['ClaimID','Provider','BeneID','total_payment','diag_expected_mean_payment','expected_vs_actual_gap','claim_to_diag_expected_mean']]
                .head(20))





#test:



### outpatient_test
# 1) Parse date columns (safe)
date_cols = outpatient_test.columns[outpatient_test.columns.str.contains('Dt|Date', case=False)]
for col in date_cols:
    outpatient_test[col] = pd.to_datetime(outpatient_test[col], errors='coerce')

today = pd.Timestamp.today()

# 2) Diagnosis / Procedure presence flags & counts
diag_cols = [f'ClmDiagnosisCode_{i}' for i in range(1, 11)]
procedure_cols = [f'ClmProcedureCode_{i}' for i in range(1, 7)]

# presence flags (1 if present, 0 if null/empty/placeholder)
placeholders = {'nan', 'none', 'unknown', 'na', '', 'nan.'}
for col in diag_cols:
    outpatient_test[col] = outpatient_test[col].astype(str).str.strip().replace({c: np.nan for c in placeholders})
    outpatient_test[col + '_present'] = outpatient_test[col].notna().astype(int)

for col in procedure_cols:
    outpatient_test[col] = outpatient_test[col].astype(str).str.strip().replace({c: np.nan for c in placeholders})
    outpatient_test[col + '_present'] = outpatient_test[col].notna().astype(int)

# totals per claim
outpatient_test['num_diagnoses'] = outpatient_test[[c + '_present' for c in diag_cols]].sum(axis=1)
outpatient_test['num_procedures'] = outpatient_test[[c + '_present' for c in procedure_cols]].sum(axis=1)

# 3) Safe numeric payments
outpatient_test['DeductibleAmtPaid'] = outpatient_test['DeductibleAmtPaid'].fillna(0).astype(float)
outpatient_test['InscClaimAmtReimbursed'] = outpatient_test['InscClaimAmtReimbursed'].fillna(0).astype(float)
outpatient_test['total_payment'] = outpatient_test['DeductibleAmtPaid'] + outpatient_test['InscClaimAmtReimbursed']

# payment flags
outpatient_test['patient_paid'] = (outpatient_test['DeductibleAmtPaid'] > 0).astype(int)
outpatient_test['insurer_paid'] = (outpatient_test['InscClaimAmtReimbursed'] > 0).astype(int)
outpatient_test['any_payment'] = ((outpatient_test['patient_paid'] == 1) | (outpatient_test['insurer_paid'] == 1)).astype(int)

# 4) Duration feature (days)
outpatient_test['claim_duration_days'] = (outpatient_test['ClaimEndDt'] - outpatient_test['ClaimStartDt']).dt.days
outpatient_test['claim_duration_days_nonzero'] = outpatient_test['claim_duration_days'].apply(lambda x: x if (pd.notna(x) and x >= 1) else 1.0)

# 5) Date anomalies relevant to outpatient_test
outpatient_test['ClaimStart_after_ClaimEnd'] = ((outpatient_test['ClaimStartDt'].notna() & outpatient_test['ClaimEndDt'].notna()) &
                                           (outpatient_test['ClaimStartDt'] > outpatient_test['ClaimEndDt'])).astype(int)

outpatient_test['ClaimStart_before_ClaimEnd'] = ((outpatient_test['ClaimStartDt'].notna() & outpatient_test['ClaimEndDt'].notna()) &
                                            (outpatient_test['ClaimStartDt'] < outpatient_test['ClaimEndDt'])).astype(int)

outpatient_test['Future_dates'] = (outpatient_test[['ClaimStartDt','ClaimEndDt']].gt(today).any(axis=1)).astype(int)

outpatient_test['ClaimStart_before_Now_but_End_missing'] = ((outpatient_test['ClaimStartDt'].notna() & outpatient_test['ClaimEndDt'].isna())).astype(int)

# conditional suspicious flags:
outpatient_test['ClaimStart_after_ClaimEnd_no_payment'] = ((outpatient_test['ClaimStart_after_ClaimEnd'] == 1) & (outpatient_test['any_payment'] == 0)).astype(int)
outpatient_test['Future_dates_no_payment'] = ((outpatient_test['Future_dates'] == 1) & (outpatient_test['any_payment'] == 0)).astype(int)

# 6) Physician features
phys_cols = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']

# freq encoding
phys_freq = pd.concat([outpatient_test[col] for col in phys_cols]).value_counts()
for col in phys_cols:
    outpatient_test[col + '_freq'] = outpatient_test[col].map(phys_freq).fillna(0).astype(int)
    outpatient_test[col + '_presence'] = outpatient_test[col].notna().astype(int)

outpatient_test['num_physicians'] = outpatient_test[[col + '_presence' for col in phys_cols]].sum(axis=1)

# 7) Diagnosis-level modeling
long = outpatient_test[['ClaimID'] + diag_cols + ['total_payment']].set_index('ClaimID').reset_index()
melted = long.melt(id_vars=['ClaimID','total_payment'], value_vars=diag_cols,
                   var_name='diag_slot', value_name='diag_code')

melted['diag_code'] = melted['diag_code'].astype(str).str.strip()
melted = melted[~melted['diag_code'].str.lower().isin(placeholders)]
melted = melted[melted['diag_code'].notna()]

diag_stats = (melted.groupby('diag_code')['total_payment']
                    .agg(['count', 'mean', 'median'])
                    .rename(columns={'mean':'mean_payment', 'median':'median_payment'})
                    .reset_index())

global_center = melted['total_payment'].median() if len(melted)>0 else 0.0
k = 10.0
diag_stats['smoothed_mean_payment'] = (diag_stats['count'] * diag_stats['mean_payment'] + k * global_center) / (diag_stats['count'] + k)

smoothed_map = diag_stats.set_index('diag_code')['smoothed_mean_payment'].to_dict()
median_map = diag_stats.set_index('diag_code')['median_payment'].to_dict()
count_map = diag_stats.set_index('diag_code')['count'].to_dict()

for col in diag_cols:
    outpatient_test[col + '_diag_exp_payment'] = outpatient_test[col].map(smoothed_map).fillna(0.0)
    outpatient_test[col + '_diag_median_payment'] = outpatient_test[col].map(median_map).fillna(0.0)
    outpatient_test[col + '_diag_count'] = outpatient_test[col].map(count_map).fillna(0).astype(int)

# aggregate
exp_cols = [c + '_diag_exp_payment' for c in diag_cols]
med_cols = [c + '_diag_median_payment' for c in diag_cols]
cnt_cols = [c + '_diag_count' for c in diag_cols]

outpatient_test['diag_expected_mean_payment'] = outpatient_test[exp_cols].replace(0, np.nan).mean(axis=1).fillna(0.0)
outpatient_test['diag_expected_median_payment'] = outpatient_test[med_cols].replace(0, np.nan).median(axis=1).fillna(0.0)
outpatient_test['diag_expected_max_payment'] = outpatient_test[exp_cols].max(axis=1).fillna(0.0)
outpatient_test['diag_expected_sum_payment'] = outpatient_test[exp_cols].sum(axis=1).fillna(0.0)
outpatient_test['diag_present_count'] = (outpatient_test[exp_cols] > 0).sum(axis=1)

outpatient_test['claim_to_diag_expected_mean'] = np.where(
    outpatient_test['diag_expected_mean_payment'] > 0,
    outpatient_test['total_payment'] / outpatient_test['diag_expected_mean_payment'],
    np.nan
)

outpatient_test['claim_to_diag_expected_max'] = np.where(
    outpatient_test['diag_expected_max_payment'] > 0,
    outpatient_test['total_payment'] / outpatient_test['diag_expected_max_payment'],
    np.nan
)

rare_threshold = 5
outpatient_test['claim_has_rare_diag'] = (outpatient_test[cnt_cols] > 0).apply(lambda row: any(row < rare_threshold), axis=1).astype(int)

outpatient_test['expected_vs_actual_gap'] = outpatient_test['total_payment'] - outpatient_test['diag_expected_mean_payment']
outpatient_test['expected_vs_actual_pct'] = outpatient_test['total_payment'] / outpatient_test['diag_expected_mean_payment'].replace({0: np.nan})

# 8) Summaries
print("Outpatient_test date/anomaly summary:")
date_issue_cols = ['ClaimStart_after_ClaimEnd','Future_dates','ClaimStart_before_ClaimEnd','ClaimStart_after_ClaimEnd_no_payment','Future_dates_no_payment']
print(outpatient_test[date_issue_cols].sum())

print("\nTop suspicious outpatient_test claims by expected_vs_actual_gap:")
print(outpatient_test.sort_values('expected_vs_actual_gap', ascending=False)
                .loc[:, ['ClaimID','Provider','BeneID','total_payment','diag_expected_mean_payment','expected_vs_actual_gap','claim_to_diag_expected_mean']]
                .head(20))













In [ ]:
print(inpatient.head(2))

In [ ]:
#aggregate
# Example aggregations – add more as needed
inpatient_provider = inpatient.groupby('Provider').agg({
    'total_payment': ['mean', 'sum', 'max', 'median'],
    'num_diagnoses': ['mean', 'max'],
    'num_procedures': ['mean', 'max'],
    'patient_paid': 'mean',
    'insurer_paid': 'mean',
    'claim_has_rare_diag': 'mean'
})

# flatten multi-level columns
inpatient_provider.columns = ['inp_' + '_'.join(col) for col in inpatient_provider.columns]
inpatient_provider.reset_index(inplace=True)

#test:

#aggregate
# Example aggregations – add more as needed
inpatient_test_provider = inpatient_test.groupby('Provider').agg({
    'total_payment': ['mean', 'sum', 'max', 'median'],
    'num_diagnoses': ['mean', 'max'],
    'num_procedures': ['mean', 'max'],
    'patient_paid': 'mean',
    'insurer_paid': 'mean',
    'claim_has_rare_diag': 'mean'
})

# flatten multi-level columns
inpatient_test_provider.columns = ['inp_' + '_'.join(col) for col in inpatient_test_provider.columns]
inpatient_test_provider.reset_index(inplace=True)



In [ ]:
outpatient_provider = outpatient.groupby('Provider').agg({
    'total_payment': ['mean', 'sum', 'max', 'median'],
    'num_diagnoses': ['mean', 'max'],
    'num_procedures': ['mean', 'max'],
    'patient_paid': 'mean',
    'insurer_paid': 'mean',
    'claim_duration_days': ['mean', 'max'],
    'claim_to_diag_expected_mean': 'mean',
    'claim_has_rare_diag': 'mean'
})

outpatient_provider.columns = ['out_' + '_'.join(col) for col in outpatient_provider.columns]
outpatient_provider.reset_index(inplace=True)

#test:

outpatient_test_provider = outpatient_test.groupby('Provider').agg({
    'total_payment': ['mean', 'sum', 'max', 'median'],
    'num_diagnoses': ['mean', 'max'],
    'num_procedures': ['mean', 'max'],
    'patient_paid': 'mean',
    'insurer_paid': 'mean',
    'claim_duration_days': ['mean', 'max'],
    'claim_to_diag_expected_mean': 'mean',
    'claim_has_rare_diag': 'mean'
})

outpatient_test_provider.columns = ['out_' + '_'.join(col) for col in outpatient_test_provider.columns]
outpatient_test_provider.reset_index(inplace=True)



In [ ]:
# merge bene data to both claim tables
inpatient_bene = inpatient.merge(benf, on='BeneID', how='left')
outpatient_bene = outpatient.merge(benf, on='BeneID', how='left')

# now aggregate per provider
beneficiary_provider = pd.concat([inpatient_bene, outpatient_bene]).groupby('Provider').agg({
    'ChronicCond_Alzheimer': 'mean',
    'ChronicCond_Heartfailure': 'mean',
    'ChronicCond_Cancer': 'mean',
    'DOB': 'count'   # number of unique patients served
})

beneficiary_provider.rename(columns={'DOB': 'num_patients'}, inplace=True)
beneficiary_provider.reset_index(inplace=True)


#test:

# merge bene data to both claim tables
inpatient_test_bene = inpatient_test.merge(benf, on='BeneID', how='left')
outpatient_test_bene = outpatient_test.merge(benf, on='BeneID', how='left')

# now aggregate per provider
beneficiary_test_provider = pd.concat([inpatient_test_bene, outpatient_test_bene]).groupby('Provider').agg({
    'ChronicCond_Alzheimer': 'mean',
    'ChronicCond_Heartfailure': 'mean',
    'ChronicCond_Cancer': 'mean',
    'DOB': 'count'   # number of unique patients served
})

beneficiary_test_provider.rename(columns={'DOB': 'num_patients'}, inplace=True)
beneficiary_test_provider.reset_index(inplace=True)



In [ ]:
# start with labels
train_labels = pd.read_csv('../data/Train-1542865627584.csv')
train_final = train_labels.copy()

# merge inpatient
train_final = train_final.merge(inpatient_provider, on='Provider', how='left')

# merge outpatient
train_final = train_final.merge(outpatient_provider, on='Provider', how='left')

# merge beneficiary-based features
train_final = train_final.merge(beneficiary_provider, on='Provider', how='left')

# fill missing values (safe approach)
train_final = train_final.fillna(0)



#test:


# start with labels

test_final = test_id.copy()

# merge inpatient
test_final = test_final.merge(inpatient_test_provider, on='Provider', how='left')

# merge outpatient
test_final = test_final.merge(outpatient_test_provider, on='Provider', how='left')

# merge beneficiary-based features
test_final = test_final.merge(beneficiary_test_provider, on='Provider', how='left')

# fill missing values (safe approach)
test_final = test_final.fillna(0)






In [ ]:
for col in train_final.columns:
    dtype = train_final[col].dtype
    
    if pd.api.types.is_numeric_dtype(train_final[col]):
        continue  # skip numeric columns
    elif pd.api.types.is_datetime64_any_dtype(train_final[col]):
        print(f"{col}: datetime → should be encoded (e.g., duration, day/month/year)")
    elif pd.api.types.is_object_dtype(train_final[col]):
        print(f"{col}: categorical → should be encoded (e.g., one-hot or label encoding)")
    else:
        print(f"{col}: unknown type {dtype} → check manually")

In [ ]:


# Encode PotentialFraud as 0/1
train_final['PotentialFraud'] = train_final['PotentialFraud'].map({'Yes': 1, 'No': 0})

# Check the result
train_final.head(1)


#test:



# Check the result
test_final.head(1)




In [ ]:
#Select columns to aggregate
agg_columns = [
    "out_claim_duration_days_mean",
    "out_claim_duration_days_max",
    "out_claim_to_diag_expected_mean_mean",
    "out_claim_has_rare_diag_mean",
    "ChronicCond_Alzheimer",
    "ChronicCond_Heartfailure",
    "ChronicCond_Cancer",
    "num_patients"
]

# Check that all columns exist
for col in agg_columns + ["Provider"]:
    if col not in train_final.columns:
        raise KeyError(f"Missing column in train_final: {col}")
    if col not in test_final.columns:
        raise KeyError(f"Missing column in test_final: {col}")

#TRAIN provider-level aggregation
train_provider = train_final.groupby("Provider").agg({
    "out_claim_duration_days_mean": "mean",
    "out_claim_duration_days_max": "max",
    "out_claim_to_diag_expected_mean_mean": "mean",
    "out_claim_has_rare_diag_mean": "mean",
    "ChronicCond_Alzheimer": "mean",
    "ChronicCond_Heartfailure": "mean",
    "ChronicCond_Cancer": "mean",
    "num_patients": "max"    # Prevent double-count & duplication across claims
}).reset_index()

#TEST provider-level aggregation
test_provider = test_final.groupby("Provider").agg({
    "out_claim_duration_days_mean": "mean",
    "out_claim_duration_days_max": "max",
    "out_claim_to_diag_expected_mean_mean": "mean",
    "out_claim_has_rare_diag_mean": "mean",
    "ChronicCond_Alzheimer": "mean",
    "ChronicCond_Heartfailure": "mean",
    "ChronicCond_Cancer": "mean",
    "num_patients": "max"
}).reset_index()

print(train_provider.shape, test_provider.shape)
train_provider.head(), test_provider.head()


In [ ]:
#merge:
train_final = train_final.merge(train_provider, on="Provider", how="left")
test_final = test_final.merge(test_provider, on="Provider", how="left")

In [ ]:
train_final = train_final.drop(columns=['Provider'])
test_final = test_final.drop(columns=['Provider'])